In [ ]:
from convokit import Corpus, Speaker, Utterance, Conversation, download
import os, pandas as pd

path = "C:/Users/L/.convokit/downloads/"
corpus = Corpus(filename=path + 'conversations-gone-awry-cmv-corpus')

def load_dfs(corpus):
    speakers = corpus.get_speakers_dataframe().drop(columns=['vectors'])
    
    conversations = corpus.get_conversations_dataframe().drop(columns=['vectors'])
    utterances = corpus.get_utterances_dataframe().drop(columns=['vectors'])
    # print(type(speakers), type(conversations), type(utterances))
    return speakers, conversations, utterances

speakers, conversations, utterances = load_dfs(corpus)

In [ ]:
# do not clean before you use. Or the data corrupt

# def delete_deleted_txt(utt_df):
#     return utt_df[utt_df['text'] != '[deleted]']
# 
# print(utterances.shape)
# utterances = delete_deleted_txt(utterances)
# print(utterances.shape)
# 
# corpus = Corpus.from_pandas(utterances_df=utterances, speakers_df= speakers, conversations_df=conversations)
# speakers, conversations, utterances = load_dfs(corpus)
# print(corpus.print_summary_stats())

In [ ]:
def print_overview(speaker_df, convo_df, utt_df):
    print("UttDf attributes:", list(utt_df.columns),'\n')
    print("ConvDf attributes:", list(convo_df.columns),'\n')
    print("SpeakerDf attributes:", list(speaker_df.columns),'\n')

    print("convo:",convo_df.shape)
    print("speaker:",speaker_df.shape)    
    print("utt:",utt_df.shape)
    
def check_column_type(df):
    r = df.sample(n=1)
    for i in df.columns:
        print(i, type(r[i].values[0]))
        
print_overview(speakers, conversations, utterances)
check_column_type(utterances)
check_column_type(conversations)

In [ ]:
import pickle

def save_df_pickle(df, path):
    df.to_pickle(path)
    print("Save to", path)
def load_df_pickle(path):
    df = pd.read_pickle(path)
    return df

In [ ]:
print(utterances.shape)
trimmed_utt = utterances[utterances['meta.score'] > 2]
print(trimmed_utt.shape)

corpus = Corpus.from_pandas(utterances_df=trimmed_utt, speakers_df= speakers, conversations_df=conversations)
print(corpus.print_summary_stats())
speakers, conversations, utterances = load_dfs(corpus)
save_df_pickle(utterances, 'utterances.pkl')
# save_corpus_pickle(corpus, 'corpus.pkl')

In [ ]:
def get_conv_id_list(corpus, condition):
    conv_id_list = []
    for conv in corpus.iter_conversations():
        if condition(conv):
            conv_id_list.append(conv.id)
    return conv_id_list

def condition(conv):
    return (len(conv.get_utterance_ids()) > 10 and len(conv.get_speaker_ids()) < 10 and conv.meta['has_removed_comment'] == False)

conv_id_list = get_conv_id_list(corpus, condition)
print(conv_id_list)

In [3]:
def rename_speaker_conv(conv):
    speakers = conv.get_speaker_ids()
    nameMap = {speaker: idx for idx, speaker in enumerate(speakers)}

    utts_df = conv.get_utterances_dataframe()
    utts_df['speakerID'] = utts_df['speaker'].apply(lambda x: nameMap.get(x, -1))

    def map_reply_to(x):
        try:
            return nameMap[utts_df.at[x, 'speaker']]
        except KeyError:
            return -1

    utts_df['reply_to_ID'] = utts_df['reply_to'].apply(map_reply_to)
    
    return utts_df

def format_conversation_user_utt_df(utt_df):
    speaker = utt_df['speakerID'].values
    text = utt_df['text'].values
    return [f"SPEAKER {speaker[i]}: {text[i]}\r" for i in range(len(speaker))]

utt_df = rename_speaker_conv(corpus.get_conversation('d27fgau'))
texts  = format_conversation_user_utt_df(utt_df)
print(texts)
print(utt_df['speaker'])
print(utt_df['reply_to_ID'])
print(utt_df['reply_to'])
print(utt_df['speakerID'])


['SPEAKER 0: If the loss of business should be incentive enough, why is there such a big push to remove discrimination laws?\n\nI always see this sort of argument brought up and I frankly don\'t understand it. It\'s taking the system we currently have and shaking it up hoping that the end result will be identical to what we\'re already dealing with. It\'s this hope and promise that the free market will just magically sort this problem out so let\'s remove governmental protections for protected classes.\n\nPretty much your only issue with the way things are currently run is that you think "it sets a bad precedent to enforce this" and I\'m not sure what exactly you mean there. Are you saying that the government is going to use this justification to...force you to hang out with gay people? I don\'t really see where this slippery slope can lead.\n\nOwning and operating a business is not really a right, it\'s a privilege. You have to get a business license, you\'re forced to pay taxes, you\

In [ ]:
def draw_combat_from_conv(conv_df):
    # a combat is a conversation pair. B replied to A, and A replied to B's utterance. A combat involves at least 3 utterances, and is a dataframe made up of utterances
    
    combat_df_list = []
    """这样写代码:
首先从dataframe的末尾往前遍历
对于每一个i, 查找'reply_to'的值, 如果为None则跳过
如果不为None, 则将i行的dataframe index和对应'reply_to'索引的行的index记录进一个临时list, 同时检查对应索引行对应的'reply_to'的值(也就是utterance的索引)的行是否存在'reply_to', 如果有则把对应索引的行的index也加入list, 直到reply_to为None. 此时将list里所有的index组合成完整的dataframe放入list
然后进入下一个i, 重复遍历直到结束"""
    ind_ls = list(reversed(conv_df.index))
    for i in ind_ls:
        if pd.isna(conv_df.at[i, 'reply_to']):
            continue
        else:
            # for j in range(_ind, len(ind_ls)):
            #     if not pd.isna(conv_df.at[i, 'reply_to'])    
            temp_list = [i, conv_df.at[i, 'reply_to']]
            try:
                a = conv_df.at[conv_df.at[temp_list[-1], 'reply_to'], 'speaker']
            except KeyError:
                # print(temp_list)
                # print("probably because you deleted some utt in a conv")
                continue
                
            while not pd.isna(conv_df.at[temp_list[-1], 'reply_to']) and conv_df.at[temp_list[-2], 'speaker'] == conv_df.at[conv_df.at[temp_list[-1], 'reply_to'], 'speaker']:                
                temp_list.append(conv_df.at[temp_list[-1], 'reply_to'])
                
            if len(temp_list) > 2:
                
                combat_df_list.append(conv_df.loc[reversed(temp_list)]['text'])
                
    return combat_df_list

# def clean_combat_df_list(df_list):
#     # remove combat_df that is included in other combat_dfs
#     # we can do it more efficiently by using speaker id hash in the function above
#     
#     # discard incomplete conversations that includes [deleted]
#     for i in range(len(df_list)):
#         if df_list[i] is not None:
#             for j in range(i+1, len(df_list)):
#                 if df_list[j] is not None:
#                     try:
#                         a = df_list[i]['text'].apply(lambda x: '[deleted]' in x).any()
#                     except KeyError:
#                         print("probably because you deleted some utt in a conv")
#                         continue
#                     if df_list[j].index.isin(df_list[i].index).all() or a:
#                         df_list[j] = None
#                     
#     return [i for i in df_list if i is not None]

def clean_combat_series_list(series_list):
    cleaned_list = []
    for i in range(len(series_list)):
        current_series = series_list[i]
        if current_series is None:
            continue

        if '[deleted]' in current_series.values:
            continue

        is_subset = any(
            current_series.index.isin(other_series.index).all() and 
            len(current_series) < len(other_series) 
            for j, other_series in enumerate(series_list) if i != j and other_series is not None
        )
        if not is_subset:
            cleaned_list.append(current_series)

    return cleaned_list

def convert_series_to_dataframe(series_list):
    dataframe_list = []
    for series in series_list:
        dataframe_list.append(pd.DataFrame(series, columns=['text']))
    return dataframe_list

def combat_df_list_from_conv_df(conv_df):
    combat_df_list = draw_combat_from_conv(conv_df)
    cleaned = clean_combat_series_list(combat_df_list)
    return convert_series_to_dataframe(cleaned)
                                       
utt_df = rename_speaker_conv(corpus.get_conversation('e1y6dui'))
# don't trim utterances according to meta_socre because it destroys the conversations
samples = ['e0xew8w', 'e1y6dui', 'e2aki76', 'e20mogq', 'e2bmhqv', 'e3r1d8u', 'e4z5qde', 'e5990cm', 'e5wygv2', 'e6b9xpa', 'e8aik19']
combat_df_list = draw_combat_from_conv(utt_df)
# cleaned = clean_combat_df_list(combat_df_list)
cleaned_1 = clean_combat_series_list(combat_df_list)
# print(utt_df)
print(combat_df_list[0])
# print(combat_df_list)
# print("\n\n\n\n",cleaned)
print("\n\n\n\n",cleaned_1)
# check_column_type(utt_df)
# check_column_type(cleaned[0])
print("\n\n\n\n",combat_df_list_from_conv_df(utt_df))
check_column_type(convert_series_to_dataframe(cleaned_1)[0])

In [ ]:
def get_combat_df_list(corpus, conv_id_list):
    combat_df_list = []
    for conv_id in conv_id_list:
        utt_df = rename_speaker_conv(corpus.get_conversation(conv_id))
        
        combat_df_list.append(combat_df_list_from_conv_df(utt_df))
    return combat_df_list

combat_df_list = get_combat_df_list(corpus, conv_id_list)
print(combat_df_list[0])
print(len(combat_df_list))

In [ ]:
import requests
def generate_answer_with_ollama(query='', context='{context not exists} ', system_prompt = 'You are a helpful assistant',input_text = None, max_tokens=1, model="llama3:latest", url="http://localhost:11434/api/generate"):
    if input_text is not None:
        input = input_text
        
    else:
        input = f"Question: {query}\nContext: {context}"

    payload = {
        'prompt': input,
        'system' : system_prompt,
        'model': model,  # Add the model parameter here
        'max_tokens': max_tokens,  # Set the maximum number of tokens to generate
        'temperature': 0.0,  # Set the temperature to 0.0 to remove randomness
        'format': 'json',
    }

    response = requests.post(url, json=payload)
    content_type = response.headers['content-type']

    try:
        # Process NDJSON response
        responses = response.content.decode('utf-8').splitlines()
        json_responses = [requests.models.complexjson.loads(line) for line in responses]
        responses = [item["response"] for item in json_responses]
        text = ''.join(responses)
        # print("NDJSON parsed successfully:", text)

        # Extract the answer from the first JSON object
        return text
    except Exception as e:
        print("Failed to decode NDJSON response:", e)
        print("Response content:", response.content)
        raise

    print("Response content:", response.content)
    raise Exception(f"Unexpected content type: {content_type}")
    

# generate_answer_with_ollama('Seperate the following text into meaning--complete sentences:\n\n', context="""Why then? Call a fucking pharmacist who can tell you what the consequences are, IF ANY. That's their job. Or pay the money to talk to a doctor on a holiday. That is what you do. Don't come on here and post the question to a bunch of people who are unqualified to answer. Get off the fucking computer and call someone. Jesus Christ.""")

In [ ]:
def judge_targeting(conversation_list):
    """
    :param conversation_list: List[Tuple[str, str]]
    :return: 0 or 1 
    """
    
    system_prompt = 'output format: {"output": OUTPUT}, where OUTPUT=0 or OUTPUT=1'
    input_text = '{"request": "judge whether the latter speaker is targeting at the former speaker\'s words.", "conversation" : '+f"{conversation_list}"+'}'
    a = generate_answer_with_ollama(system_prompt=system_prompt, input_text=input_text, max_tokens=1)
    
    return a

# conversations_example = [('Speaker A', "Hello how can I help you? Do you think sex is sin?"), ('Speaker B', "Fuck you")]
# for i in range (12): print(judge_targeting(conversations_example))

In [ ]:
def single_text_summarization(text):
    system_prompt = 'output format: {"summarization": SUMM}, where SUMM is the summarization of the text '
    
    input_text = '{"request": "summarize the reddit speaker\'s emotions and key propositions, perspectives", "SpeakerText" : '+f"{text}"+'}'
    
    a = generate_answer_with_ollama(system_prompt=system_prompt, input_text=input_text, max_tokens=min(len(text)/3, 30))
    
    return a




In [3]:
from convokit import Corpus, Speaker, Utterance, Conversation, download  
import os, pandas as pd
import requests

def load_dfs(corpus):  
    speakers = corpus.get_speakers_dataframe().drop(columns=['vectors'])  
    
    conversations = corpus.get_conversations_dataframe().drop(columns=['vectors'])  
    utterances = corpus.get_utterances_dataframe().drop(columns=['vectors'])  
    # print(type(speakers), type(conversations), type(utterances))  
    return speakers, conversations, utterances

def print_overview(speaker_df, convo_df, utt_df):  
    print("UttDf attributes:", list(utt_df.columns),'\n')  
    print("ConvDf attributes:", list(convo_df.columns),'\n')  
    print("SpeakerDf attributes:", list(speaker_df.columns),'\n')  
  
    print("convo:",convo_df.shape)  
    print("speaker:",speaker_df.shape)      
    print("utt:",utt_df.shape)  
      
def check_column_type(df):  
    r = df.sample(n=1)  
    for i in df.columns:  
        print(i, type(r[i].values[0]))

def get_conv_id_list(corpus, condition):  
    conv_id_list = []  
    for conv in corpus.iter_conversations():  
        if condition(conv):  
            conv_id_list.append(conv.id)  
    return conv_id_list  
  
def condition(conv):  
    return (len(conv.get_utterance_ids()) > 10 and len(conv.get_speaker_ids()) < 10 and conv.meta['has_removed_comment'] == False)

def condition(conv):  
    return True

def rename_speaker_conv(conv):  
    speakers = conv.get_speaker_ids()  
    nameMap = {speaker: idx for idx, speaker in enumerate(speakers)}  
  
    utts_df = conv.get_utterances_dataframe()  
    utts_df['speakerID'] = utts_df['speaker'].apply(lambda x: nameMap.get(x, -1))  
  
    def map_reply_to(x):  
        try:  
            return nameMap[utts_df.at[x, 'speaker']]  
        except KeyError:  
            return -1  
  
    utts_df['reply_to_ID'] = utts_df['reply_to'].apply(map_reply_to)  
      
    return utts_df  
  
def format_conversation_user_utt_df(utt_df):  
    speaker = utt_df['speakerID'].values  
    text = utt_df['text'].values  
    return [f"SPEAKER {speaker[i]}: {text[i]}\r" for i in range(len(speaker))]

def draw_combat_from_conv(conv_df):
    # a combat is a conversation pair. B replied to A, and A replied to B's utterance. A combat involves at least 3 utterances, and is a dataframe made up of utterances
    
    combat_df_list = []
    """这样写代码:
首先从dataframe的末尾往前遍历
对于每一个i, 查找'reply_to'的值, 如果为None则跳过
如果不为None, 则将i行的dataframe index和对应'reply_to'索引的行的index记录进一个临时list, 同时检查对应索引行对应的'reply_to'的值(也就是utterance的索引)的行是否存在'reply_to', 如果有则把对应索引的行的index也加入list, 直到reply_to为None. 此时将list里所有的index组合成完整的dataframe放入list
然后进入下一个i, 重复遍历直到结束"""
    ind_ls = list(reversed(conv_df.index))
    for i in ind_ls:
        if pd.isna(conv_df.at[i, 'reply_to']):
            continue
        else:
            # for j in range(_ind, len(ind_ls)):
            #     if not pd.isna(conv_df.at[i, 'reply_to'])    
            temp_list = [i, conv_df.at[i, 'reply_to']]
            try:
                a = conv_df.at[conv_df.at[temp_list[-1], 'reply_to'], 'speaker']
            except KeyError:
                # print(temp_list)
                # print("probably because you deleted some utt in a conv")
                continue
                
            while not pd.isna(conv_df.at[temp_list[-1], 'reply_to']) and conv_df.at[temp_list[-2], 'speaker'] == conv_df.at[conv_df.at[temp_list[-1], 'reply_to'], 'speaker']:                
                temp_list.append(conv_df.at[temp_list[-1], 'reply_to'])
                
            if len(temp_list) > 2:
                
                combat_df_list.append(conv_df.loc[reversed(temp_list)]['text'])
                
    return combat_df_list

def clean_combat_series_list(series_list):  
    cleaned_list = []  
    for i in range(len(series_list)):  
        current_series = series_list[i]  
        if current_series is None:  
            continue  
  
        if '[deleted]' in current_series.values:  
            continue  
  
        is_subset = any(  
            current_series.index.isin(other_series.index).all() and   
len(current_series) < len(other_series)   
            for j, other_series in enumerate(series_list) if i != j and other_series is not None  
        )  
        if not is_subset:  
            cleaned_list.append(current_series)  
  
    return cleaned_list  
  
def convert_series_to_dataframe(series_list):  
    dataframe_list = []  
    for series in series_list:  
        dataframe_list.append(pd.DataFrame(series, columns=['text']))  
    return dataframe_list  
  
def combat_df_list_from_conv_df(conv_df):  
    combat_df_list = draw_combat_from_conv(conv_df)  
    cleaned = clean_combat_series_list(combat_df_list)  
    return convert_series_to_dataframe(cleaned)

def get_combat_df_list(corpus, conv_id_list):  
    combat_df_list = []  
    for conv_id in conv_id_list:  
        utt_df = rename_speaker_conv(corpus.get_conversation(conv_id))  
          
        combat_df_list.append(combat_df_list_from_conv_df(utt_df))  
    return combat_df_list

import requests  

def generate_answer_with_ollama(query='', context='{context not exists} ', system_prompt = 'You are a helpful assistant',input_text = None, max_tokens=1, model="llama3:latest", url="http://localhost:11434/api/generate"):  
    if input_text is not None:  
        input = input_text  
          
    else:  
        input = f"Question: {query}\nContext: {context}"  
  
    payload = {  
        'prompt': input,  
        'system' : system_prompt,  
        'model': model,  # Add the model parameter here  
        'max_tokens': max_tokens,  # Set the maximum number of tokens to generate  
        'temperature': 0.0,  # Set the temperature to 0.0 to remove randomness  
        # 'format': 'json',  
    }  
  
    response = requests.post(url, json=payload, stream=False)  
  
    try:  
        # Process NDJSON response  
        responses = response.content.decode('utf-8').splitlines()  
        json_responses = [requests.models.complexjson.loads(line) for line in responses]  
        responses = [item["response"] for item in json_responses]  
        text = ''.join(responses)  
        # print("NDJSON parsed successfully:", text)  
        return text
  
        # Extract the answer from the first JSON object        return text  
    except Exception as e:  
        print("Failed to decode NDJSON response:", e)  
        print("Response content:", response.content)  
        raise 
  
    # print("Response content:", response.content)  

def judge_targeting(conversation_pair):  
    """  
    :param conversation_pair: Tuple[str, str]    :return: 0 or 1   
    """  
    system_prompt = 'output format: {"output": OUTPUT}, where OUTPUT=0 or OUTPUT=1'  
    input_text = '{"request": "judge whether the latter speaker is targeting at the former speaker\'s words. Targeting includes opposing opinions, verbal violence", "conversation" : '+f"{conversation_pair}"+'}'  
    a = generate_answer_with_ollama(system_prompt=system_prompt, input_text=input_text, max_tokens=1)  
      
    return a

def single_text_summarization(text):  
    system_prompt = 'output format: {"summarization": SUMM}, where SUMM is the summarization of the text '  
    input_text = '{"request": "summarize the reddit speaker\'s emotions and key propositions, perspectives", "SpeakerText" : '+f"{text}"+'}'  
    a = generate_answer_with_ollama(system_prompt=system_prompt, input_text=input_text, max_tokens=min(len(text)/3, 30))  
      
    return a

def sentence_immiatation(text):  
    system_prompt = 'output format: {"immitation": IMM}, where IMM is a sentence-by-sentence immiatation of the original text'  
    input_text = '{"request": "immitate the reddit speaker\'s every sentence, expression", "SpeakerText" : '+f"{text}"+'}'  
    a = generate_answer_with_ollama(system_prompt=system_prompt, input_text=input_text, max_tokens=min(len(text), 30))  
      
    return a



In [ ]:
os.environ['http_proxy'] = 'http://localhost:7890'
os.environ['https_proxy'] = 'http://localhost:7890'
path = "C:/Users/L/.convokit/downloads/"
corpus = Corpus(filename=path + 'conversations-gone-awry-cmv-corpus')
conv_id_list = get_conv_id_list(corpus, condition)
combat_df_list = get_combat_df_list(corpus, conv_id_list)

In [1]:
# print(combat_df_list[0][0])
# print(type(combat_df_list[0][0]))

# for conv in combat_df_list:
#     for i in conv:
#         i['agu_1'] = i['text'].apply(lambda x: single_text_summarization(x))
#         print(i['agu_1'].values)
#         break
#     break
    
import pickle    

def save_combat_df_list_to_pickle(combat_df_list, path):
    with open(path, 'wb') as f:
        pickle.dump(combat_df_list, f)
    print("Save to", path)
    
def load_combat_df_list_from_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
    

In [22]:
def condition(conv):  
    return True

conv_id_list = get_conv_id_list(corpus, condition)
combat_df_list = get_combat_df_list(corpus, conv_id_list)
save_combat_df_list_to_pickle(combat_df_list, 'combat_df_list_full.pkl')

Save to combat_df_list_full.pkl


In [23]:
ls = [
]
for i in load_combat_df_list_from_pickle('combat_df_list_full.pkl'):
    for j in i:
        ls.append(j)
        
concat = pd.concat(ls)
print(concat.shape)

(35116, 1)


In [5]:
print(load_combat_df_list_from_pickle('combat_df_list_full.pkl')[0][0])

                                                      text
id                                                        
cue9wmd  Wow, that's quite a claim. Can you substantiat...
cueiwqd  &gt;Nigeria is the economic capital of Africa,...
cuek32r  The Nigerian GDP is 21st largest in the world,...
cuevw78  Lol ok, seventh most populated country in the ...


In [6]:
def sentence_immiatation(text):  
    system_prompt = 'OUTPUT JSON FORMAT:\n{"--IMMITATION--": YOUR_IMMITATION_TEXTS}\n\nYour immitation should preserve the Speaker\'s semantic meaning and emotions sentence by sentence.\n'  
    input_text = '{'+ "Speaker's Text: "  +f"{text}"+'}'  
    a = generate_answer_with_ollama(system_prompt=system_prompt, input_text=input_text, max_tokens=min(len(text), 30))  
      
    return a

for conv in combat_df_list:
    for i in conv:
        i['imm_1'] = i['text'].apply(lambda x: sentence_immiatation(x))
        print(i['imm_1'].values)
        break
    break
    

['{"--IMMITATION--": {"Speaker\'s Text: Wow, that\'s quite a claim. Can you substantiate it? \\nNigeria is the economic capital of Africa, with premier educational facilities as well as one of the fastest growing economies on earth. \\nIf your premise were true then you would see higher educated persons from every \\"destitute\\" nation, but that\'s not the case. \\nyou are providing no legitimate basis for your opinions, you should consider doing research on the subject before you further make your opinions public."}'
 '{"--IMMITATION--": {"I\'m shocked and disagree with that statement. Nigeria being the economic capital of Africa is far from the truth. I think you\'re mistaking it for other countries like Tunisia, Egypt, South Africa, or Angola which are indeed prominent economies in Africa."}}'
 '{"--IMMITATION--": "I think there may be some confusion here. According to the World Bank, Nigeria\'s GDP is actually around 27th largest in the world, not 21st. This means it\'s larger tha

In [4]:
def sentence_immiatation(text):  
    system_prompt = 'OUTPUT FORMAT:\n{"--IMMITATION--": YOUR_IMMITATION_TEXTS}\n\nYour immitation should preserve the Speaker\'s semantic meaning and emotions sentence by sentence.\n'  
    input_text = 'Speaker\'s Text:'  +f"{text}"  
    a = generate_answer_with_ollama(system_prompt=system_prompt, input_text=input_text, max_tokens=min(len(text), 30))  
      
    return a

def check_immitation_successful(combat_df, col_name='imm', signal='{"immitation":', possible_signal='": '):
    # check if indicator is True, then format

    def check_imm(x):
        # signal for immitation failure
        return signal in x
    
    combat_df[col_name+'_check'] = combat_df[col_name].apply(check_imm)
    print(combat_df[col_name+'_check'].values)
    
    def format_json(x):
        if signal in x:
            result = x.split(signal)[1].split('}')[0]
            if possible_signal in result:
                parts = result.split(possible_signal)
                if len(parts) > 1:
                    return parts[1]
                else:
                    return parts[0]  # or handle the case where possible_signal is at the end
            return result
        return x
    combat_df[col_name] = combat_df.apply(lambda row: format_json(row[col_name]) if row[col_name+'_check'] else row['text'], axis=1)
    return combat_df

def check(path):
    combat_df_list = load_combat_df_list_from_pickle(path)
    for ind, conv in enumerate(combat_df_list):
        # if ind<=176:
        #     continue
        if conv:
            for i in conv:
                i['imm'] = i['text'].apply(lambda x: sentence_immiatation(x))
                print(i['text'].values)
                print(i['imm'].values)
            
                i = check_immitation_successful(i, col_name='imm', signal='--IMMITATION--',possible_signal='": ')
                
                print(i['imm'].values)
                # print(i['imm_check'].values)
    
                # break
            break
        else:
            print("empty")
        
check('combat_df_list_full.pkl')        

['Twin studies studies suggest that about 80 percent of variance in IQ is genetic.'
 'Sorry Perpetualjoke, your comment has been removed: \n\n&gt; Comment Rule 5\\. "No low effort comments. Comments that are only jokes, links, or \'written upvotes\', for example. Humor and affirmations of agreement can be contained within more substantial comments." [See the wiki page for more information.](http://www.reddit.com/r/changemyview/wiki/rules#wiki_rule_5)\n\nIf you would like to appeal, please [message the moderators by clicking this link.](http://www.reddit.com/message/compose?to=%2Fr%2Fchangemyview&amp;subject=Removed+Comment+Rule+5+Post+Appeal&amp;message=Perpetualjoke+would+like+to+appeal+the+removal+of+[his/her+post](https://www.reddit.com/r/changemyview/comments/3i8mfw/cmv_i_believe_that_black_people_are_less/cuedywn\\))'
 'Are you kidding me?You decided to remove it after 5 months?'
 'You decided to edit it after 5 months.  Why would we leave up an empty comment?'
 "I just can't beli

In [51]:
def add_immitation_and_save(combat_df_list):
    le = len(combat_df_list)
    for ind, conv in enumerate(combat_df_list):
        print("process conv started"+f"{ind}/{le}")
        # if ind<=176:
        #     continue
        
        for i in conv:
            i['imm_1'] = i['text'].apply(lambda x: sentence_immiatation(x))
            i = check_immitation_successful(i, col_name='imm_1', signal='--IMMITATION--',possible_signal='": ')
        if ind % 10 == 0 and ind > 0:
            save_combat_df_list_to_pickle(combat_df_list, f"combat_df_list_imms_1_full_{ind-10}-{ind}.pkl")
    save_combat_df_list_to_pickle(combat_df_list, 'combat_df_list_imms_1_full.pkl')
    print("done")
    
ori = load_combat_df_list_from_pickle('combat_df_list_full.pkl')
add_immitation_and_save(ori)


process conv started0/6842
[ True  True  True  True  True  True]
process conv started1/6842
[ True  True  True  True]
process conv started2/6842
[ True  True  True  True False]
process conv started3/6842
[ True  True  True  True  True  True]
process conv started4/6842
[ True  True  True  True  True  True  True  True]
process conv started5/6842
[ True  True  True  True  True  True  True  True]
process conv started6/6842
process conv started7/6842
[ True  True  True]
[ True  True  True]
process conv started8/6842
[ True  True  True]
process conv started9/6842
[ True  True  True  True]
process conv started10/6842
[ True  True  True  True]
[ True  True  True]
Save to combat_df_list_imms_1_full_0-10.pkl
process conv started11/6842
[ True  True  True  True]
[ True  True  True]
process conv started12/6842
[ True  True  True]
process conv started13/6842
process conv started14/6842
[ True  True  True  True  True  True  True  True  True]
process conv started15/6842
[ True  True  True]
[ True  Tr

ChunkedEncodingError: Response ended prematurely

In [5]:
def add_immitation_and_save(combat_df_list):
    le = len(combat_df_list)
    for ind, conv in enumerate(combat_df_list):
        print("process conv started"+f"{ind}/{le}")
        if ind<=5320:
            continue
        
        for i in conv:
            i['imm_1'] = i['text'].apply(lambda x: sentence_immiatation(x))
            i = check_immitation_successful(i, col_name='imm_1', signal='--IMMITATION--',possible_signal='": ')
        if ind % 10 == 0 and ind > 0:
            save_combat_df_list_to_pickle(combat_df_list, f"combat_df_list_imms_1_full_{ind-10}-{ind}.pkl")
    save_combat_df_list_to_pickle(combat_df_list, 'combat_df_list_imms_1_full.pkl')
    print("done")
    
conti = load_combat_df_list_from_pickle('combat_df_list_imms_1_full_5310-5320.pkl')
add_immitation_and_save(conti)

process conv started0/6842
process conv started1/6842
process conv started2/6842
process conv started3/6842
process conv started4/6842
process conv started5/6842
process conv started6/6842
process conv started7/6842
process conv started8/6842
process conv started9/6842
process conv started10/6842
process conv started11/6842
process conv started12/6842
process conv started13/6842
process conv started14/6842
process conv started15/6842
process conv started16/6842
process conv started17/6842
process conv started18/6842
process conv started19/6842
process conv started20/6842
process conv started21/6842
process conv started22/6842
process conv started23/6842
process conv started24/6842
process conv started25/6842
process conv started26/6842
process conv started27/6842
process conv started28/6842
process conv started29/6842
process conv started30/6842
process conv started31/6842
process conv started32/6842
process conv started33/6842
process conv started34/6842
process conv started35/6842
pr

In [7]:
combat_df_list_imm = load_combat_df_list_from_pickle('combat_df_list_imms_1_full.pkl')

print(combat_df_list_imm[3000][0])

                                                      text  \
id                                                           
dwbyfjo  Parkour isn’t even viable for self defense my ...   
dwdafzs  I mean, Krav Maga is good enough for the Israe...   
dwdam3g  It is though. Because close quarters combat an...   
dwdb7rr  Stories like Jhoon Rhee, grand master of Tae K...   
dwdbjjl  First off, a grandmaster of any martial art is...   
dwdcbpm  Also, he was a master at the time. It was the ...   

                                                     imm_1  imm_1_check  
id                                                                       
dwbyfjo  \n\nParkour isn't even a viable option for sel...         True  
dwdafzs  "I'm saying that since Krav Maga has been effe...         True  
dwdam3g  "I understand why you'd think that, but it's t...         True  
dwdb7rr  "You're saying stories about grand masters get...         True  
dwdbjjl  "Listen, being a grandmaster in any martial ar... 

In [8]:
print(combat_df_list_imm[1][0]['text'].values)
print(combat_df_list_imm[1][0]['imm_check'].values)


["Do you even read the posts you reply to? \n\nIt's also bullshit. Investment happens when consumers spend money, and a basic income funded by a wealth tax would increase spending at virtually every level of the economy."
 "My apologies, I should've read your post better. Still, I think it's doubtful that you could control all wealth export like that. Wouldn't it be possible to send goods/wealth oversea while keeping it under the radar?"
 "Possible, yes, but corporations are already forced to keep extensive financial records, and 'losing' a lot of money from your public books isn't really attractive to investors."
 "It is an interesting idea. Why do you reckon they haven't implemented it yet?"]
[ True  True False False]


In [28]:
print(sentence_immiatation("Possible, yes, but corporations are already forced to keep extensive financial records, and 'losing' a lot of money from your public books isn't really attractive to investors."))

{"--IMMITATION--": "Indeed, it's likely that companies would face challenges in concealing significant financial losses as they're already required to maintain detailed accounting records. The prospect of misreporting substantial losses wouldn't be appealing to shareholders."}


In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk


# 假设dataframe的格式如下：
# df = pd.DataFrame({
#     'original_conversation': ['Sentence 1', 'Sentence 2', ...],
#     'enhanced_conversation': ['Enhanced Sentence 1', 'Enhanced Sentence 2', ...]
# })

df = combat_df_list_imm[0][0].copy()
# 加载停用词
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# 文本预处理函数
def preprocess(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

# 对原始对话和增强对话进行预处理
df['text_processed'] = df['text'].apply(preprocess)
df['imm_processed'] = df['imm'].apply(preprocess)

# 合并所有对话文本
all_conversations = pd.concat([df['text_processed'], df['imm_processed']])

# 创建词袋模型
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(all_conversations)

# 为原始对话和增强对话赋予权重
original_weight = 2
enhanced_weight = 1
weights = np.concatenate([np.full(len(df), original_weight), np.full(len(df), enhanced_weight)])

# 进行加权
X_weighted = X.multiply(weights[:, np.newaxis])

# 训练LDA模型
n_topics = 10  # 假设主题数为10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X_weighted)

# 输出主题和主题词
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

n_top_words = 10
tf_feature_names = vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, n_top_words)


Topic #0:
cum penis balls extra packages chromosomes masculine traditional sex characteristics
Topic #1:
traditional masculine sex characteristics biological ancestry wrong norms certain physical
Topic #2:
masculine traditional able feminine butch woman without within range pursuits
Topic #3:
traditional masculine sex characteristics biological ancestry wrong norms certain physical
Topic #4:
traditional masculine sex characteristics biological ancestry wrong norms certain physical
Topic #5:
must also understood knew long human chimp even like know
Topic #6:
saying stereotypes racial manly fact race fit different regardless male
Topic #7:
male chromosomes would longer accident balls penis characteristics sex someone
Topic #8:
traditional masculine sex characteristics biological ancestry wrong norms certain physical
Topic #9:
traditional masculine sex characteristics biological ancestry wrong norms certain physical


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\L\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\L\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import time

# 加载停用词
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# 假设combat_df_list_imm是一个包含334个DataFrame的列表
# 每个DataFrame的格式如下：
# df = pd.DataFrame({'text': ['utterance 1', 'utterance 2', ...]})

def preprocess(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

# 合并每个DataFrame的'text'列为一个文档
documents = []
for sublist in combat_df_list_imm:
    combined_text = ' '.join([preprocess(utt) for df in sublist for utt in df['text']])
    documents.append(combined_text)

# 创建词袋模型
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# 训练LDA模型
n_topics = 10  # 假设主题数为10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# 输出主题和主题词
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

n_top_words = 10
tf_feature_names = vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, n_top_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\L\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\L\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Topic #0:
people gt like would think even media one news say
Topic #1:
gt would people think like one parents point said make
Topic #2:
people would gt like think someone one say even saying
Topic #3:
people would gt think like one get us make eat
Topic #4:
gt women people men would like think one gender get
Topic #5:
gt right people government law would rights free speech like
Topic #6:
people would guns gun like gt make one get could
Topic #7:
woman gender women men trans man would sex gt like
Topic #8:
gt would dog people one like also think consent make
Topic #9:
people gt black like white think one would race know


In [15]:
n_topics_list = list(range(1,20))
perplexity_results = {}

# 迭代不同的n_topics值，并评估每个模型的困惑度
for i, n_topics in enumerate(n_topics_list):
    start_time = time.time()
    print(f"Processing {i+1}/{len(n_topics_list)}: n_topics = {n_topics}")
    
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(X)
    
    # 评估困惑度
    perplexity = lda.perplexity(X)
    perplexity_results[n_topics] = perplexity
    
    end_time = time.time()
    print(f"n_topics = {n_topics} completed. Perplexity = {perplexity:.2f}. Time taken: {end_time - start_time:.2f} seconds\n")

# 输出所有n_topics的困惑度
print("Perplexity results for different n_topics:")
for n_topics, perplexity in perplexity_results.items():
    print(f"n_topics = {n_topics}, Perplexity = {perplexity:.2f}")

# 输出最优的n_topics
best_n_topics = min(perplexity_results, key=perplexity_results.get)
print(f"\nBest n_topics by Perplexity: {best_n_topics} with Perplexity = {perplexity_results[best_n_topics]:.2f}")

Processing 1/19: n_topics = 1
n_topics = 1 completed. Perplexity = 3342.06. Time taken: 0.08 seconds

Processing 2/19: n_topics = 2
n_topics = 2 completed. Perplexity = 3249.84. Time taken: 1.25 seconds

Processing 3/19: n_topics = 3
n_topics = 3 completed. Perplexity = 3184.77. Time taken: 0.98 seconds

Processing 4/19: n_topics = 4
n_topics = 4 completed. Perplexity = 3159.51. Time taken: 0.89 seconds

Processing 5/19: n_topics = 5
n_topics = 5 completed. Perplexity = 3159.10. Time taken: 0.92 seconds

Processing 6/19: n_topics = 6
n_topics = 6 completed. Perplexity = 3141.25. Time taken: 0.90 seconds

Processing 7/19: n_topics = 7
n_topics = 7 completed. Perplexity = 3127.62. Time taken: 0.89 seconds

Processing 8/19: n_topics = 8
n_topics = 8 completed. Perplexity = 3130.44. Time taken: 0.90 seconds

Processing 9/19: n_topics = 9
n_topics = 9 completed. Perplexity = 3127.95. Time taken: 1.01 seconds

Processing 10/19: n_topics = 10
n_topics = 10 completed. Perplexity = 3108.57. Tim

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import time
import pickle

# 加载停用词
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# 假设combat_df_list_imm是一个包含子列表的列表，每个子列表包含DataFrame
# 每个DataFrame的格式如下：
# df = pd.DataFrame({'text': ['utterance 1', 'utterance 2', ...]})

def preprocess(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

# 合并每个子列表中的DataFrame的'text'列为一个文档
documents = []
for sublist in combat_df_list_imm:
    combined_text = ' '.join([preprocess(utt) for df in sublist for utt in df['text']])
    documents.append(combined_text)

# 创建词袋模型
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# 设置不同的n_topics值进行迭代
n_topics_list = list(range(1, 21))
results = []

# 迭代不同的n_topics值，并保存每个模型的词袋、主题词等参数
for i, n_topics in enumerate(n_topics_list):
    start_time = time.time()
    print(f"Processing {i+1}/{len(n_topics_list)}: n_topics = {n_topics}")
    
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=100)
    lda.fit(X)
    
    # 保存结果
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        topics.append([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])
    
    result = {
        'n_topics': n_topics,
        'model': lda,
        'topics': topics,
        'perplexity': lda.perplexity(X),
        'vectorizer': vectorizer
    }
    
    results.append(result)
    
    end_time = time.time()
    print(f"n_topics = {n_topics} completed. Perplexity = {result['perplexity']:.2f}. Time taken: {end_time - start_time:.2f} seconds\n")

# 使用pickle保存结果
with open('lda_results.pkl', 'wb') as f:
    pickle.dump(results, f)

print("Results have been saved to lda_results.pkl")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\L\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\L\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processing 1/20: n_topics = 1
n_topics = 1 completed. Perplexity = 3342.06. Time taken: 0.65 seconds

Processing 2/20: n_topics = 2
n_topics = 2 completed. Perplexity = 3146.57. Time taken: 7.92 seconds

Processing 3/20: n_topics = 3
n_topics = 3 completed. Perplexity = 3102.17. Time taken: 6.72 seconds

Processing 4/20: n_topics = 4
n_topics = 4 completed. Perplexity = 3093.10. Time taken: 5.99 seconds

Processing 5/20: n_topics = 5
n_topics = 5 completed. Perplexity = 3131.21. Time taken: 6.54 seconds

Processing 6/20: n_topics = 6
n_topics = 6 completed. Perplexity = 3109.15. Time taken: 6.29 seconds

Processing 7/20: n_topics = 7
n_topics = 7 completed. Perplexity = 3090.35. Time taken: 6.36 seconds

Processing 8/20: n_topics = 8
n_topics = 8 completed. Perplexity = 3081.04. Time taken: 6.52 seconds

Processing 9/20: n_topics = 9
n_topics = 9 completed. Perplexity = 3080.02. Time taken: 7.01 seconds

Processing 10/20: n_topics = 10
n_topics = 10 completed. Perplexity = 3047.37. Tim

In [17]:
# 合并每个子列表中的DataFrame的'text'列为一个文档
documents = []
for sublist in combat_df_list_imm:
    combined_text = ' '.join([preprocess(utt) for df in sublist for utt in df['imm']])
    documents.append(combined_text)

# 创建词袋模型
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# 设置不同的n_topics值进行迭代
n_topics_list = list(range(1, 21))
results = []

# 迭代不同的n_topics值，并保存每个模型的词袋、主题词等参数
for i, n_topics in enumerate(n_topics_list):
    start_time = time.time()
    print(f"Processing {i+1}/{len(n_topics_list)}: n_topics = {n_topics}")
    
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=100)
    lda.fit(X)
    
    # 保存结果
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        topics.append([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])
    
    result = {
        'n_topics': n_topics,
        'model': lda,
        'topics': topics,
        'perplexity': lda.perplexity(X),
        'vectorizer': vectorizer
    }
    
    results.append(result)
    
    end_time = time.time()
    print(f"n_topics = {n_topics} completed. Perplexity = {result['perplexity']:.2f}. Time taken: {end_time - start_time:.2f} seconds\n")

# 使用pickle保存结果
with open('lda_results_imm.pkl', 'wb') as f:
    pickle.dump(results, f)

print("Results have been saved to lda_results_imm.pkl")


Processing 1/20: n_topics = 1
n_topics = 1 completed. Perplexity = 3549.44. Time taken: 0.71 seconds

Processing 2/20: n_topics = 2
n_topics = 2 completed. Perplexity = 3414.56. Time taken: 9.58 seconds

Processing 3/20: n_topics = 3
n_topics = 3 completed. Perplexity = 3391.73. Time taken: 7.99 seconds

Processing 4/20: n_topics = 4
n_topics = 4 completed. Perplexity = 3416.91. Time taken: 6.79 seconds

Processing 5/20: n_topics = 5
n_topics = 5 completed. Perplexity = 3464.32. Time taken: 6.96 seconds

Processing 6/20: n_topics = 6
n_topics = 6 completed. Perplexity = 3467.46. Time taken: 6.72 seconds

Processing 7/20: n_topics = 7
n_topics = 7 completed. Perplexity = 3485.88. Time taken: 6.87 seconds

Processing 8/20: n_topics = 8
n_topics = 8 completed. Perplexity = 3506.67. Time taken: 7.28 seconds

Processing 9/20: n_topics = 9
n_topics = 9 completed. Perplexity = 3558.25. Time taken: 7.87 seconds

Processing 10/20: n_topics = 10
n_topics = 10 completed. Perplexity = 3500.36. Tim

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

with open('lda_results.pkl', 'rb') as f:
    results_1 = pickle.load(f)
    
with open('lda_results_imm.pkl', 'rb') as f:
    results_2 = pickle.load(f)
# 比较相同主题数量下的主题词
for result_1, result_2 in zip(results_1, results_2):
    n_topics = result_1['n_topics']
    topics_1 = result_1['topics']
    topics_2 = result_2['topics']

    print(f"Number of Topics: {n_topics}")
    print("Dataset 1 Topics:")
    for i, topic in enumerate(topics_1):
        print(f"  Topic #{i+1}: {', '.join(topic)}")

    print("Dataset 2 Topics:")
    for i, topic in enumerate(topics_2):
        print(f"  Topic #{i+1}: {', '.join(topic)}")
    print("\n")


Number of Topics: 1
Dataset 1 Topics:
  Topic #1: people, gt, would, like, think, one, even, make, get, point
Dataset 2 Topics:
  Topic #1: people, like, would, think, one, get, even, someone, right, gt


Number of Topics: 2
Dataset 1 Topics:
  Topic #1: people, gt, would, think, like, right, one, get, even, make
  Topic #2: people, gt, would, like, think, one, women, men, even, person
Dataset 2 Topics:
  Topic #1: people, like, would, think, women, one, even, someone, get, gt
  Topic #2: people, like, would, think, get, right, one, even, gt, someone


Number of Topics: 3
Dataset 1 Topics:
  Topic #1: people, gt, would, like, think, right, one, get, government, even
  Topic #2: gt, people, would, one, think, like, child, make, point, person
  Topic #3: people, gt, would, like, think, women, one, men, someone, gender
Dataset 2 Topics:
  Topic #1: people, like, would, women, think, get, one, someone, even, gt
  Topic #2: people, like, would, think, right, get, one, even, someone, gt
  To

In [23]:
from collections import Counter

def get_top_words(model, vectorizer, n_top_words=10):
    feature_names = vectorizer.get_feature_names_out()
    top_words = []
    for topic_idx, topic in enumerate(model.components_):
        top_words.extend([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    return top_words

# 假设results_1和results_2包含LDA模型和对应的vectorizer
all_top_words = []

for result in results_1 + results_2:
    top_words = get_top_words(result['model'], result['vectorizer'])
    all_top_words.extend(top_words)

# 统计词频
word_freq = Counter(all_top_words)

# 找出那些出现在80%以上主题中的词
n_topics_total = len(results_1 + results_2)
threshold = 0.8 * n_topics_total

common_words = [word for word, freq in word_freq.items() if freq > threshold]
print("Common words to be removed:", common_words)


Common words to be removed: ['people', 'gt', 'would', 'like', 'think', 'one', 'even', 'make', 'get', 'point', 'right', 'women', 'men', 'government', 'child', 'someone', 'us', 'saying', 'law', 'speech', 'free']


In [24]:
common_words = ['people', 'gt', 'would', 'like', 'think','one', 'even', 'make', 'get']

# 加载原始停用词
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 更新停用词词典
updated_stop_words = stop_words.union(set(common_words))

# 重新进行预处理并进行LDA分析
def preprocess(text, stop_words):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

# 合并每个子列表中的DataFrame的'text'列为一个文档
documents = []
for sublist in combat_df_list_imm:
    combined_text = ' '.join([preprocess(utt, updated_stop_words) for df in sublist for utt in df['text']])
    documents.append(combined_text)

# 创建词袋模型
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# 设置不同的n_topics值进行迭代
n_topics_list = list(range(1, 21))
results = []

# 迭代不同的n_topics值，并保存每个模型的词袋、主题词等参数
for i, n_topics in enumerate(n_topics_list):
    start_time = time.time()
    print(f"Processing {i+1}/{len(n_topics_list)}: n_topics = {n_topics}")
    
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=100)
    lda.fit(X)
    
    # 保存结果
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        topics.append([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])
    
    result = {
        'n_topics': n_topics,
        'model': lda,
        'topics': topics,
        'perplexity': lda.perplexity(X),
        'vectorizer': vectorizer
    }
    
    results.append(result)
    
    end_time = time.time()
    print(f"n_topics = {n_topics} completed. Perplexity = {result['perplexity']:.2f}. Time taken: {end_time - start_time:.2f} seconds\n")

# 使用pickle保存结果
with open('lda_results_updated.pkl', 'wb') as f:
    pickle.dump(results, f)

print("Results have been saved to lda_results_updated.pkl")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\L\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing 1/20: n_topics = 1
n_topics = 1 completed. Perplexity = 3966.04. Time taken: 0.63 seconds

Processing 2/20: n_topics = 2
n_topics = 2 completed. Perplexity = 3702.30. Time taken: 6.99 seconds

Processing 3/20: n_topics = 3
n_topics = 3 completed. Perplexity = 3615.10. Time taken: 6.77 seconds

Processing 4/20: n_topics = 4
n_topics = 4 completed. Perplexity = 3632.29. Time taken: 5.49 seconds

Processing 5/20: n_topics = 5
n_topics = 5 completed. Perplexity = 3659.95. Time taken: 5.74 seconds

Processing 6/20: n_topics = 6
n_topics = 6 completed. Perplexity = 3653.98. Time taken: 5.58 seconds

Processing 7/20: n_topics = 7
n_topics = 7 completed. Perplexity = 3676.92. Time taken: 5.86 seconds

Processing 8/20: n_topics = 8
n_topics = 8 completed. Perplexity = 3684.30. Time taken: 5.90 seconds

Processing 9/20: n_topics = 9
n_topics = 9 completed. Perplexity = 3675.41. Time taken: 6.17 seconds

Processing 10/20: n_topics = 10
n_topics = 10 completed. Perplexity = 3678.08. Tim

In [1]:
def analyze_print_pickle_results(path= 'lda_results.pkl', resultss = None):
    
    import pickle
    # 加载保存的LDA结果
    with open(path, 'rb') as f:
        results = pickle.load(f)
    results = resultss if resultss is not None else results
    
    # 分析保存的参数
    for result in results:
        print(f"Number of Topics: {result['n_topics']}")
        print(f"Perplexity: {result['perplexity']:.2f}")
        print("Top 10 words per topic:")
        for idx, topic in enumerate(result['topics']):
            print(f"  Topic #{idx + 1}: {', '.join(topic)}")
        print("\n")
    
    # 选择最佳模型
    best_result = min(results, key=lambda x: x['perplexity'])
    print(f"Best Model with {best_result['n_topics']} topics")
    print(f"Perplexity: {best_result['perplexity']:.2f}")
    print("Top 10 words per topic:")
    for idx, topic in enumerate(best_result['topics']):
        print(f"Best Topic Words#{idx}: {', '.join(topic)}")
    
    # 使用最佳模型进一步分析
    best_model = best_result['model']
    vectorizer = best_result['vectorizer']
    
    # 示例：获取某个文档的主题分布
    example_document = "This is an example document to analyze with the LDA model."
    example_vectorized = vectorizer.transform([example_document])
    example_topic_distribution = best_model.transform(example_vectorized)
    print("Example Document Topic Distribution:")
    print(example_topic_distribution)
    
    # 示例：提取某个主题的详细信息
    topic_idx = 0
    topic_words = best_model.components_[topic_idx]
    top_words_idx = topic_words.argsort()[:-11:-1]
    top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_idx]
    print(f"Top words for topic #{topic_idx + 1}: {', '.join(top_words)}")
analyze_print_pickle_results('lda_results_updated.pkl')

Number of Topics: 1
Perplexity: 3966.04
Top 10 words per topic:
  Topic #1: point, right, say, way, want, also, someone, could, person, saying


Number of Topics: 2
Perplexity: 3702.30
Top 10 words per topic:
  Topic #1: right, point, government, us, want, also, say, way, trump, could
  Topic #2: women, men, person, say, way, someone, point, saying, something, could


Number of Topics: 3
Perplexity: 3615.10
Top 10 words per topic:
  Topic #1: government, could, want, point, gun, money, guns, also, work, need
  Topic #2: women, men, person, say, point, way, gender, black, woman, could
  Topic #3: right, trump, say, point, us, way, saying, also, want, someone


Number of Topics: 4
Perplexity: 3632.29
Top 10 words per topic:
  Topic #1: government, gun, could, want, guns, money, work, capitalism, system, also
  Topic #2: women, men, say, could, also, black, way, white, point, woman
  Topic #3: right, trump, say, us, point, way, law, free, speech, want
  Topic #4: gender, point, trans, som

In [32]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pickle

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return [scores['pos'], scores['neg'], scores['neu'], scores['compound']]

processed_vadar_imm = []
for sublist in combat_df_list_imm:
    processed_sublist = []
    for df in sublist:
        df['txt_sentiment'] = df['text'].apply(analyze_sentiment)
        df['imm_sentiment'] = df['imm'].apply(analyze_sentiment)
        processed_sublist.append(df)
    processed_vadar_imm.append(processed_sublist)
    

with open('processed_vadar_imm.pkl', 'wb') as f:
    pickle.dump(processed_vadar_imm, f)
    
for sublist in processed_vadar_imm:
    for df in sublist:
        print(df['txt_sentiment'].values)
        print(df['text'].values)
        print(df['imm_sentiment'].values)
        break
    break
    


[list([0.04, 0.092, 0.868, -0.6548]) list([0.036, 0.046, 0.918, -0.2263])
 list([0.0, 0.0, 1.0, 0.0]) list([0.027, 0.129, 0.844, -0.8451])
 list([0.048, 0.05, 0.902, -0.0433])]
['&gt;So within our predefined two genders, we have a huge range of variability. Is a "tomboy" type woman "crazy" for thinking that the hyper feminine stereotype of women is stupid, and that she should be able to do "manly" things without being called butch?\n\n\nI never said anything like that. In fact I\'ve said the opposite. If someone wants to be a tomboy, whatever, go for it. That\'s individualism. But saying you\'re a boy because you act manly according to stereotypes when you\'re factually female is the same as saying you\'re a different race because you fit racial stereotypes regardless of the fact that you have non of that racial blood. It\'s gross. '
 "&gt; But saying you're a boy because you act manly according to stereotypes when you're factually female is the same as saying you're a different race b

In [33]:
import pandas as pd
from textblob import TextBlob
import pickle

# 假设combat_df_list_imm是一个包含子列表的列表，每个子列表包含DataFrame
# 每个DataFrame的格式如下：
# df = pd.DataFrame({'text': ['utterance 1', 'utterance 2', ...]})

# 定义情感分析函数
def analyze_sentiment(text):
    blob = TextBlob(text)
    return [blob.sentiment.polarity, blob.sentiment.subjectivity]

with open('processed_vadar_imm.pkl', 'rb') as f:
    processed_vadar_imm = pickle.load(f)
    
# 处理每个DataFrame
processed_textblob_imm = []
for sublist in processed_vadar_imm:
    processed_sublist = []
    for df in sublist:
        df['txt_blob'] = df['text'].apply(analyze_sentiment)
        df['imm_blob'] = df['imm'].apply(analyze_sentiment)
        processed_sublist.append(df)
    processed_textblob_imm.append(processed_sublist)

# 使用pickle保存处理后的结果
with open('processed_textblob_imm.pkl', 'wb') as f:
    pickle.dump(processed_textblob_imm, f)

print("Results have been saved to processed_textblob_imm.pkl")

def print_sentiment_results(processed_vadar_imm):
    for sublist in processed_vadar_imm:
        for df in sublist:
            print(df.sample(n=2))
            break
        break
        
print_sentiment_results(processed_textblob_imm)

Results have been saved to processed_textblob_imm.pkl
                                                      text  \
id                                                           
cvsxaph  Like I said a million times before. Exceptions...   
cvs92ym  &gt;So within our predefined two genders, we h...   

                                                     agu_1  \
id                                                           
cvsxaph  {"summarization": "The speaker is likely frust...   
cvs92ym  {"summarization": "The speaker expresses frust...   

                                                       imm  imm_check  \
id                                                                      
cvsxaph  Like I said a million times before. Exceptions...      False   
cvs92ym   "So, within our established binary gender fra...       True   

                         txt_sentiment                  imm_sentiment  \
id                                                                      
cvsxaph  